In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("./admissions.csv.gz", compression='gzip')

In [3]:
df1=pd.read_csv("./discharge.csv.gz", compression='gzip')

In [4]:
#해당 정보를 datetime형식으로 바꿈
df['admittime'] = pd.to_datetime(df['admittime'])
df['dischtime'] = pd.to_datetime(df['dischtime'])

df.sort_values(by=['subject_id', 'admittime'], inplace=True)

#환자가 다음 방문하는 시기를 나타냄
df['next_admittime'] = df.groupby(['subject_id'])['admittime'].shift(-1)

# 재입원하는 시기를 계산
df['days_to_next_admit'] = (df['next_admittime'] - df['dischtime']).dt.days

# 재입원 기간이 30일이 넘어가면 1로 아니면 0으로 매핑
df['readmission_30'] = np.where(df['days_to_next_admit'] <= 30, 1, 0)

df.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,next_admittime,days_to_next_admit,readmission_30
266356,10000019,25058216,2129-05-21 19:16:00,2129-05-23 18:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0,NaT,NaN,0
434288,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0,2180-06-26 18:27:00,50.0,0
446284,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0,2180-07-23 12:35:00,25.0,1
427806,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0,2180-08-05 23:44:00,11.0,1
453525,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0,NaT,NaN,0


In [5]:
#필요한 feature만 남김
df=df[['subject_id','hadm_id','admittime','dischtime','readmission_30']]
df=df.dropna()
df.head()

,subject_id,hadm_id,admittime,dischtime,readmission_30
266356,10000019,25058216,2129-05-21 19:16:00,2129-05-23 18:30:00,0
434288,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,0
446284,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,1
427806,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,1
453525,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,0


In [6]:
#필요한 feature만 남김
df1=df1[['subject_id','hadm_id','charttime','storetime','text']]
df1=df1.dropna()
df1.head()

,subject_id,hadm_id,charttime,storetime,text
0,10000032,22595853,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032,22841357,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032,29079034,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
3,10000032,25742920,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...
4,10000084,23052089,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...


In [7]:
df1['charttime'] = pd.to_datetime(df1['charttime'])
df1['storetime'] = pd.to_datetime(df1['storetime'])

# 테이블을 'dischtime'와 'charttime'로 정렬
df.sort_values(by=['dischtime'], inplace=True)
df1.sort_values(by=['charttime'], inplace=True)

# merge_asof를 사용하여 조인
# 'direction'을 'forward'로 설정하여 dischtime 이후 가장 가까운 charttime을 찾아서 조인.
joined_data = pd.merge_asof(df1, df, by=['subject_id', 'hadm_id'], 
                            left_on='charttime', right_on='dischtime',
                            direction='forward')

In [8]:
#결측 데이터 제거및 필요한 feature 선택
joined_data=joined_data.dropna()
joined_data=joined_data[['text','readmission_30']]

In [10]:
results=joined_data

In [11]:
#Discharge_Dianosis에 대한 정보만 사용
def Discharge_Diagnosis(text):
    start_keyword = "Discharge Diagnosis:"
    end_keyword = "Discharge Condition:"
    start_index = text.find(start_keyword)

    if start_index != -1:
        start_index += len(start_keyword)  
        end_index = text.find(end_keyword, start_index)

        Discharge_Diagnosis = text[start_index:end_index] if end_index != -1 else text[start_index:]
    else:
        Discharge_Diagnosis = ""

    return Discharge_Diagnosis
results['text']=results['text'].apply(Discharge_Diagnosis)

In [12]:
#텍스트 데이터 전처리
results.dropna(subset=['text'], inplace=True)
results['text'] = results['text'].apply(lambda x: x.replace('\n', ''))
results['text'] = results['text'].apply(lambda x: x.replace('.', ''))
results = results[results['text'] != '']
results=results.reset_index(drop=True)

In [13]:
sample_df = results.sample(n=110000)

In [14]:
sample_df = sample_df.reset_index(drop=True)

#100,000개와 10,000개로 데이터를 나눈다.
small_df = df.iloc[:10000]  
large_df = df.iloc[10000:]   

small_df = small_df.reset_index(drop=True)
large_df = large_df.reset_index(drop=True)

In [21]:
small_df.to_csv('./test.csv',index=False)
large_df.to_csv('./comback_rev.csv',index=False)